In [1]:
# 1) Temizle (eski sürümleri kaldır)
%pip uninstall -y transformers peft trl accelerate bitsandbytes datasets evaluate soundfile librosa -q

# 2) PyTorch + temel araçlar
%pip install "torch>=2.4.0" tensorboard -q

# 3) Transformers (Gemma için önerilen)
%pip install "transformers>=4.51.3" -q

# 4) HF ekosistemi + TRL + PEFT (Google Gemma QLoRA rehberindeki sürümler)
%pip install --upgrade -q \
  "datasets==3.3.2" \
  "accelerate==1.4.0" \
  "evaluate==0.4.3" \
  "bitsandbytes==0.45.3" \
  "trl==0.21.0" \
  "peft==0.14.0" \
  protobuf \
  sentencepiece \
  soundfile


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour

In [ ]:

from google.colab import drive
drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import transformers, peft, trl
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    EarlyStoppingCallback,
    TrainerCallback,
)
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, PeftModel
from trl import SFTTrainer

print("Transformers:", transformers.__version__)
print("PEFT:", peft.__version__)
print("TRL:", trl.__version__)


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Transformers: 4.57.3
PEFT: 0.14.0
TRL: 0.21.0


In [8]:
#Huggingface'e bağlanma
from huggingface_hub import login
login()

In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

base_model_id = "google/gemma-2-2b-it"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    use_fast=True,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
)

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
model.config.use_cache = False  # gradient checkpointing ile çakışmaması için

print("✅ Base Gemma 2B yüklendi (henüz LoRA yok)")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Base Gemma 2B yüklendi (henüz LoRA yok)


In [19]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],   # Gemma için hafif ve stabil
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

print("✅ LoRA konfigürasyonu hazır (SFTTrainer içinde kullanılacak)")


✅ LoRA konfigürasyonu hazır (SFTTrainer içinde kullanılacak)


In [20]:
from datasets import load_dataset

data_url = "https://raw.githubusercontent.com/guzelTemli/YapayZekaChatbot/refs/heads/main/FINE-TUNE/QA_dataset_Vson.jsonl"

raw_ds = load_dataset("json", data_files={"train": data_url}, split="train")

# Train / validation ayır
dataset = raw_ds.train_test_split(test_size=0.1, seed=42)
dataset["validation"] = dataset["test"]
del dataset["test"]

print(dataset)
print("Train örnek sayısı:", len(dataset["train"]))
print("Validation örnek sayısı:", len(dataset["validation"]))
print("Örnek text:\n", dataset["train"][0]["text"][:400])


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 937
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 105
    })
})
Train örnek sayısı: 937
Validation örnek sayısı: 105
Örnek text:
 <start_of_turn>user
Soru: VPN istemcisinin güncel tutulması neden zorunludur?
<end_of_turn>
<start_of_turn>model
Eski sürümlerde güvenlik açıkları olabilir ve saldırganlar bu zafiyetleri kullanarak sisteme sızabilir.
<end_of_turn>


In [21]:
from transformers import TrainingArguments, EarlyStoppingCallback, TrainerCallback
from trl import SFTTrainer
import inspect
import os
import shutil

# ✅ ANA CHECKPOINT DİZİNİ
checkpoint_dir = "/content/drive/MyDrive/gemma-2b-train-qlora-checkpoints"

# ✅ EN İYİ MODELİN AYRI KAYDEDİLECEĞİ DİZİN
best_model_dir = "/content/drive/MyDrive/gemma-2b-train-qlora-best"
os.makedirs(best_model_dir, exist_ok=True)

# ✅ BEST MODELİ OTOMATİK KOPYALAYAN CALLBACK
class SaveBestModelCallback(TrainerCallback):
    def on_train_end(self, args, state, control, **kwargs):
        if state.best_model_checkpoint is not None:
            print(f"✅ En iyi checkpoint bulundu:\n{state.best_model_checkpoint}")
            print("📦 Best model ayrı klasöre kopyalanıyor...")

            if os.path.exists(best_model_dir):
                shutil.rmtree(best_model_dir)

            shutil.copytree(state.best_model_checkpoint, best_model_dir)
            print("✅ Best model başarıyla kaydedildi:", best_model_dir)
        else:
            print("⚠️ Best model checkpoint bulunamadı!")

# ✅ EĞİTİM AYARLARI
desired_args = dict(
    output_dir=checkpoint_dir,

    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,   # efektif batch = 8

    num_train_epochs=5,              # ✅ 5 EPOCH
    learning_rate=2e-4,
    warmup_steps=50,

    bf16=True,                       # ✅ L4/T4 için uygun
    fp16=False,

    logging_steps=50,

    # ✅ CHECKPOINT KAYDI VAR
    save_strategy="steps",
    save_steps=200,

    # ✅ EVALUATION VAR (EarlyStopping için şart)
    # 🔴 DÜZELTİLDİ: eval_strategy DEĞİL, evaluation_strategy
    eval_strategy="steps",
    eval_steps=200,

    load_best_model_at_end=True,     # ✅ En iyi checkpoint RAM’e yüklenir
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    report_to="none"
)

# ✅ Güvenli parametre filtresi
sig = inspect.signature(TrainingArguments)
valid_keys = set(sig.parameters.keys())
filtered_args = {k: v for k, v in desired_args.items() if k in valid_keys}

training_args = TrainingArguments(**filtered_args)

print("✅ TrainingArguments hazır")

# ✅ EARLY STOPPING
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.0
)

# ✅ TRAINER
trainer = SFTTrainer(
    model=model,                      # BASE GEMMA (LoRA henüz uygulanmadı)
    args=training_args,
    train_dataset=dataset["train"],   # 👈 bir önceki bloktan
    eval_dataset=dataset["validation"],
    peft_config=lora_config,          # 👈 LoRA config burada veriliyor
    callbacks=[early_stopping, SaveBestModelCallback()]  # ✅ İKİ CALLBACK AKTİF
)

train_result = trainer.train()
print(train_result)


✅ TrainingArguments hazır


Adding EOS to train dataset:   0%|          | 0/937 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/937 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/937 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/105 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/105 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/105 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 1, 'pad_token_id': 0}.


Step,Training Loss,Validation Loss
200,1.452400,1.554299
400,1.122100,1.520783


✅ En iyi checkpoint bulundu:
/content/drive/MyDrive/gemma-2b-train-qlora-checkpoints/checkpoint-400
📦 Best model ayrı klasöre kopyalanıyor...
✅ Best model başarıyla kaydedildi: /content/drive/MyDrive/gemma-2b-train-qlora-best
TrainOutput(global_step=590, training_loss=1.4806621486857785, metrics={'train_runtime': 602.4951, 'train_samples_per_second': 7.776, 'train_steps_per_second': 0.979, 'total_flos': 4370807869083648.0, 'train_loss': 1.4806621486857785})


In [22]:
# LoRA adapter + tokenizer kaydı
adapter_dir = "/content/gemma-2b-train-qlora-final"
os.makedirs(adapter_dir, exist_ok=True)

trainer.model.save_pretrained(adapter_dir)
tokenizer.save_pretrained(adapter_dir)

print("✅ Final LoRA adapter + tokenizer kaydedildi:", adapter_dir)


✅ Final LoRA adapter + tokenizer kaydedildi: /content/gemma-2b-train-qlora-final


In [32]:
!zip -r gemma_qlora_adapter.zip /content/gemma-2b-train-qlora-final


  adding: content/gemma-2b-train-qlora-final/ (stored 0%)
  adding: content/gemma-2b-train-qlora-final/tokenizer_config.json (deflated 96%)
  adding: content/gemma-2b-train-qlora-final/README.md (deflated 66%)
  adding: content/gemma-2b-train-qlora-final/chat_template.jinja (deflated 52%)
  adding: content/gemma-2b-train-qlora-final/special_tokens_map.json (deflated 70%)
  adding: content/gemma-2b-train-qlora-final/tokenizer.json (deflated 84%)
  adding: content/gemma-2b-train-qlora-final/tokenizer.model (deflated 51%)
  adding: content/gemma-2b-train-qlora-final/adapter_config.json (deflated 53%)
  adding: content/gemma-2b-train-qlora-final/adapter_model.safetensors (deflated 8%)


In [25]:
def ask_model(question: str) -> str:
    messages = [
        {"role": "user", "content": question}
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,  # modelden cevap üretmesini söyle
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=300,
            do_sample=True,
            temperature=0.3,
            top_p=0.9,
        )

    answer_ids = outputs[0][inputs["input_ids"].shape[-1]:]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True)
    return answer.strip()


In [26]:
print(ask_model("Uzaktan çalışma sırasında VPN neden zorunlu tutulmalıdır?"))


Çalışan cihazleri kurum ağında değilse, veri sızabilir; VPN güvenli bağlantı sağlar.


In [12]:
!pip install -U bitsandbytes transformers


In [5]:
# 1) Repo'yu klonla
!git clone https://github.com/guzelTemli/YapayZekaChatbot.git

Cloning into 'YapayZekaChatbot'...
remote: Enumerating objects: 290, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 290 (delta 9), reused 18 (delta 5), pack-reused 268 (from 2)
Receiving objects: 100% (290/290), 573.88 MiB | 26.61 MiB/s, done.
Resolving deltas: 100% (144/144), done.


In [ ]:
#Huggingface'e bağlanma
from huggingface_hub import login
login()

In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

BASE_MODEL_ID = "google/gemma-2-2b-it"
ADAPTER_PATH  = "/content/YapayZekaChatbot/FINE-TUNE/gemma-2b-train-qlora-final"  # kendi ortamına göre değiştir

# 1) Tokenizer'ı adapter klasöründen al (çünkü chat_template ve özel tokenlar sende)
tokenizer = AutoTokenizer.from_pretrained(ADAPTER_PATH, use_fast=True)

# 2) Base Gemma modelini YÜKSEK HASSASİYETLE yükle (4-bit YOK!)
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    device_map="auto",          # Colab GPU'daysan bu yeterli
    torch_dtype=torch.bfloat16  # veya torch.float16 / torch.float32
)

# 3) LoRA adapter'ını base modelin üzerine giydir
model = PeftModel.from_pretrained(
    base_model,
    ADAPTER_PATH,
)

# 4) PAD/EOS hizalama (eğitimde ne yaptıysak aynısı)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
model.eval()

print("✅ Fine-tune edilmiş Gemma 2B modeli (LoRA adaptörlü) başarıyla yüklendi!")


`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

✅ Fine-tune edilmiş Gemma 2B modeli (LoRA adaptörlü) başarıyla yüklendi!


In [20]:
def ask_model(question: str) -> str:
    messages = [{"role": "user", "content": question}]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,   # modelden cevap istiyoruz
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        do_sample=True,
        temperature=0.3,
        top_p=0.9,
        )

    # Sadece cevabı al (prompt kısmını kes)
    answer_ids = outputs[0][inputs["input_ids"].shape[-1]:]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True)
    return answer.strip()


In [21]:
print(ask_model("Uzaktan çalışma sırasında VPN neden zorunlu tutulmalıdır?"))


Kurum ağına bağlanmayan kullanıcılar hassas verileri şifresiz olarak paylaşabilir; VPN bu tür riskleri engeller.
